In [92]:
import numpy as np
from keras.models import load_model
from pythainlp.tokenize import word_tokenize
from thai_tokenizer import Tokenizer
from pythainlp import word_vector
from keras.models import Model, load_model
import numpy as np

In [93]:
# Read input.txt into separate lists
with open("input.txt", "r", encoding="utf-8-sig") as input_file:
    input_data = [line.strip().split("::")[1] for line in input_file]

tokenizer = Tokenizer()

# Pythai Tokenize
words = [
    [w for w in word_tokenize(s, engine="newmm", keep_whitespace=False)]
    for s in input_data
]

print(len(words))
# Print the tokenized words
for sentence in words:
    print(sentence)

FileNotFoundError: [Errno 2] No such file or directory: 'input.txt'

In [ ]:
vocab = set([w for s in words for w in s])
print('Vocab size = '+str(len(vocab)))
print(vocab)

Vocab size = 4608
{'กรรมพันธุ์', 'ถึงกับ', '~', 'โดยด่วน', 'คนเรา', 'กรีด', 'ท่าจะ', 'ปุ๊', 'มองตาม', 'นิดหน่อย', 'ต้น', 'เข้า', 'ถึงแม้ว่า', 'เมื่อปีก่อน', 'สบาย', 'ข้างหน้า', 'ฟัง', 'ฝูง', '700', 'บาร์', 'แป้ง', 'ของขวัญ', 'นัก', 'อัด', 'มิ', 'ฮ่า', 'ศูนย์', 'ปลาทู', 'น้น', 'เมตตา', 'วัยชรา', 'เด็กทารก', 'ฉััน', 'ด่วน', 'อยู่ดีๆ', 'ป่าน', 'ภูมิใจ', 'ยอม', 'ญึ่ปุ่น', 'หัวใจ', 'Meter', 'นาย', 'โด', 'พัน', 'แฟลช', 'ปีก่อน', 'นับ', 'พันธุกรรม', 'จดหมาย', 'เป็นงาน', 'กระรอก', 'ตามใจชอบ', 'ทรมาน', '!', 'พระพุทธเจ้า', '.', ')', 'นู้น', 'ให้หน้า', 'ขอ', 'สลบ', 'แสง', 'ราคา', 'เป็นห่วง', 'ก่อนกำหนด', 'ฝ้า', 'ค้นหา', 'นักแสดง', 'หย่อน', 'จำลอง', 'ช่าง', 'รับประกัน', 'อ้วก', 'ความดี', 'ตาตี่', 'ราชา', 'สะดุดตา', 'สุขุม', 'ทำผิด', 'มัน', 'เป็นครั้งแรก', 'รสชาติ', 'สนามเด็กเล่น', 'ขาย', 'ทำสวน', 'เริด', 'ยาง', 'ประมาณ', 'การรักษา', 'ผ่านสายตา', 'ศิลป์', 'สลิ่ม', 'ใคร', 'อุ้ย', 'เปลวไฟ', 'ลูกปืน', 'สะอาด', 'รักษาตัว', 'สนาม', 'ทำให้', 'ปลดล็อก', 'บรรยากาศ', 'ชัง', 'ตามนั้น', 'เกิด', 'วิ่งหนี', 'หร

In [ ]:
#Word Vector Test
word_vector_length = 300
wv = word_vector.WordVector(model_name="thai2fit_wv").get_model() # load thai2fit_wv from pythainlp

max_sentence_length = 34
print('Input Line',len(words))
print('Max Sentence Length',max_sentence_length)

input = np.zeros((len(words),max_sentence_length,word_vector_length))
sample_count = 0
for s in words[:len(words)]:
    word_count = 0
    for w in s:
        try:
            input[sample_count,max_sentence_length-word_count-1,:] = wv[w]
            word_count = word_count+1
        except:
            pass
    sample_count = sample_count+1

# print(input)
print(input.shape)

Input Line 4818
Max Sentence Length 34
(4818, 34, 300)


In [ ]:
class_mapping = {0: 'E', 1: 'P', 2: 'T'}

# Load the model
model = load_model('model.h5')
print(model.summary())

pred = model.predict(input)

pred_labels = []
for i in range(len(pred[0])):
    output1_argmax = pred[0][i].argmax()
    output1_label = class_mapping[output1_argmax] if output1_argmax in class_mapping else ''

    output2_argmax = pred[1][i].argmax()
    output2_label = class_mapping[output2_argmax] if output2_argmax in class_mapping else ''

    output3_argmax = pred[2][i].argmax()
    output3_label = class_mapping[output3_argmax] if output3_argmax in class_mapping else ''

    # Combine the predicted labels
    predicted_labels = [label for label in [output1_label, output2_label, output3_label] if label != '']
    predicted_labels_str = ",".join(predicted_labels)

    pred_labels.append(f"{i + 1}::{predicted_labels_str}")

# Save predictions to a text file
with open('result.txt', 'w') as f:
    f.write("\n".join(pred_labels))


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 34, 300)]    0           []                               
                                                                                                  
 bidirectional_3 (Bidirectional  (None, 34, 128)     186880      ['input_2[0][0]']                
 )                                                                                                
                                                                                                  
 bidirectional_4 (Bidirectional  (None, 128)         98816       ['bidirectional_3[0][0]']        
 )                                                                                                
                                                                                              